In [1]:
### Import packages ###
import os
import re
import json
import pickle
import argparse
import numpy as np
import pandas as pd

# Input Datatype

In [5]:
DataTypeInput = "COMPAS"
PredictorTypeInput = "TreeFARMS"

In [ ]:
PassiveLearning_MTTreeFarmsRashomonNum11.pkl
RashomonQBC_MTTreeFarmsRashomonNum11.pkl
RashomonQBC_MTTreeFarmsRashomonNum1010.pkl
RashomonQBC_MTTreeFarmsRashomonNum100100.pkl

In [7]:
### Set Up  ###
cwd = os.getcwd()
ResultsDirectory = os.path.join(cwd, "Results",DataTypeInput, PredictorTypeInput)
SaveDirectory = os.path.join(ResultsDirectory, "ProcessedResults")
AllSelectorMethodErrors = {selector: [] for selector in json.loads(args.SelectorType)}

#### Construct Directory ###
Directory = os.path.join(ResultsDirectory, "Raw")


NameError: name 'args' is not defined

In [8]:
import os
import shutil

# Define the base directory containing the files
base_dir = "/Users/simondn/Documents/RashomonActiveLearning/Results/COMPAS/TreeFarms/Raw"

# Define the categories based on file endings
categories = [
    "PassiveLearning_MTTreeFarmsRashomonNum11.pkl",
    "RashomonQBC_MTTreeFarmsRashomonNum11.pkl",
    "RashomonQBC_MTTreeFarmsRashomonNum1010.pkl",
    "RashomonQBC_MTTreeFarmsRashomonNum100100.pkl",
]

# Create directories for each category
for category in categories:
    category_dir = os.path.join(base_dir, category.split(".")[0])  # Strip '.pkl' for directory name
    os.makedirs(category_dir, exist_ok=True)

# Iterate over files and move them to the appropriate category
for filename in os.listdir(base_dir):
    if filename.endswith(".pkl"):
        for category in categories:
            if filename.endswith(category):
                src = os.path.join(base_dir, filename)
                dest = os.path.join(base_dir, category.split(".")[0], filename)
                shutil.move(src, dest)
                print(f"Moved: {filename} -> {dest}")
                break


Moved: Seed7_DataCOMPAS_TP0.2_CP0.8_STRashomonQBC_MTTreeFarmsRashomonNum100100.pkl -> /Users/simondn/Documents/RashomonActiveLearning/Results/COMPAS/TreeFarms/Raw/RashomonQBC_MTTreeFarmsRashomonNum100100/Seed7_DataCOMPAS_TP0.2_CP0.8_STRashomonQBC_MTTreeFarmsRashomonNum100100.pkl
Moved: Seed5_DataCOMPAS_TP0.2_CP0.8_STRashomonQBC_MTTreeFarmsRashomonNum100100.pkl -> /Users/simondn/Documents/RashomonActiveLearning/Results/COMPAS/TreeFarms/Raw/RashomonQBC_MTTreeFarmsRashomonNum100100/Seed5_DataCOMPAS_TP0.2_CP0.8_STRashomonQBC_MTTreeFarmsRashomonNum100100.pkl
Moved: Seed3_DataCOMPAS_TP0.2_CP0.8_STRashomonQBC_MTTreeFarmsRashomonNum100100.pkl -> /Users/simondn/Documents/RashomonActiveLearning/Results/COMPAS/TreeFarms/Raw/RashomonQBC_MTTreeFarmsRashomonNum100100/Seed3_DataCOMPAS_TP0.2_CP0.8_STRashomonQBC_MTTreeFarmsRashomonNum100100.pkl
Moved: Seed1_DataCOMPAS_TP0.2_CP0.8_STRashomonQBC_MTTreeFarmsRashomonNum100100.pkl -> /Users/simondn/Documents/RashomonActiveLearning/Results/COMPAS/TreeFarms/R